In [1]:
from google.colab import drive

# 구글 드라이브 마운트
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import os
import json
import random
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report

In [7]:
# 레이블 파일 로딩
with open('/content/labels_updated.json', 'r') as file:
    labels = json.load(file)

# 사용할 감정 리스트
valid_emotions = ["sadness", "happy", "anger", "panic"]

# 데이터와 라벨을 저장할 리스트
data = []
label_list = []

# 이미지 파일 로딩 및 필터링
for item in labels:
    img_path = item['image']
    emotion = item['emotion']

    # 유효한 감정인지 확인
    if emotion in valid_emotions:
        # 이미지 로딩 및 전처리
        img_path = img_path.replace("C:\\Users\\user\\Desktop\\Final Project\\Images Data\\ESTSOFT_Cropped\\img\\", "/content/drive/My Drive/img/")
        image = load_img(img_path, target_size=(224, 224))
        image = img_to_array(image)
        image = preprocess_input(image)

        # 리스트에 추가
        data.append(image)
        label_list.append(emotion)

# 데이터가 로드되었는지 확인
print(f"Loaded {len(data)} images with labels.")

# 데이터가 존재하는지 확인
if len(data) == 0 or len(label_list) == 0:
    raise ValueError("No data or labels loaded. Please check the dataset.")

# 리스트를 넘파이 배열로 변환
data = np.array(data)
label_list = np.array(label_list)

# 라벨 이진화 (One-hot encoding)
lb = LabelBinarizer()
label_list = lb.fit_transform(label_list)

# 학습 데이터셋과 검증 데이터셋 분리
X_train, X_val, y_train, y_val = train_test_split(data, label_list, test_size=0.2, random_state=42)

# VGG16 모델 불러오기 (ImageNet 가중치 사용, 최상위 Fully Connected Layer 제외)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# 최상위 Layer 추가
x = base_model.output
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
predictions = Dense(len(lb.classes_), activation='softmax')(x)

# 새로운 모델 정의
model = Model(inputs=base_model.input, outputs=predictions)

# 최상위 Layer만 학습
for layer in base_model.layers:
    layer.trainable = False

# 모델 컴파일
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# 모델 학습
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=32)

# 검증 데이터셋에 대한 예측
y_pred = model.predict(X_val)

# y_pred와 y_val의 차원 확인 후 변환
if len(y_pred.shape) > 1 and y_pred.shape[1] > 1:
    y_pred_classes = np.argmax(y_pred, axis=1)
else:
    y_pred_classes = y_pred  # 차원이 1인 경우 직접 사용

if len(y_val.shape) > 1 and y_val.shape[1] > 1:
    y_true_classes = np.argmax(y_val, axis=1)
else:
    y_true_classes = y_val  # 차원이 1인 경우 직접 사용

# 성능 평가
print(classification_report(y_true_classes, y_pred_classes, target_names=lb.classes_))


Loaded 5994 images with labels.
Epoch 1/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 20s 119ms/step - accuracy: 0.3908 - loss: 9.2486 - val_accuracy: 0.5063 - val_loss: 1.0958
Epoch 2/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 15s 98ms/step - accuracy: 0.7113 - loss: 0.7096 - val_accuracy: 0.6464 - val_loss: 0.9246
Epoch 3/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 15s 97ms/step - accuracy: 0.8971 - loss: 0.3050 - val_accuracy: 0.6505 - val_loss: 1.1199
Epoch 4/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 14s 96ms/step - accuracy: 0.9661 - loss: 0.1208 - val_accuracy: 0.6847 - val_loss: 1.0585
Epoch 5/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 14s 95ms/step - accuracy: 0.9936 - loss: 0.0420 - val_accuracy: 0.6764 - val_loss: 1.2113
Epoch 6/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 14s 95ms/step - accuracy: 0.9989 - loss: 0.0144 - val_accuracy: 0.6689 - val_loss: 1.4566
Epoch 7/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 14s 96ms/step - accuracy: 0.9996 - loss: 0.0114 - val_accuracy: 0.6839 - val_loss: 1.3299
Epoch 8/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 14s 96ms/step - a